# Converting the PDF files into vector database  

# 1. Initial setup
## 1.1. Imports
This setup includes loading environment variables from a `.env` file, setting the required environment variables, and importing the necessary modules for further processing. It ensures that the code has access to the required APIs and functions for the subsequent tasks.

In [2]:
# Initial imports
from dotenv import load_dotenv
import glob
import os
from IPython.display import display, Markdown
import chromadb

# Load the variables from .env file and set the API key (or user may manually set the API key)
load_dotenv()  

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["ANTHROPIC_API_KEY"] = os.getenv('ANTHROPIC_API_KEY')
os.environ["MATHPIX_API_ID"] = os.getenv('MATHPIX_API_KEY')
#openai.api_key = os.getenv('OPENAI_API_KEY')

# Langchain framework
from langchain import hub
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel # for RAG with source
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain_experimental.text_splitter import SemanticChunker
from langchain.text_splitter import RecursiveCharacterTextSplitter

## The following loaders are used for options
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.document_loaders import PyPDFium2Loader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.document_loaders import MathpixPDFLoader
from langchain_community.document_loaders import PDFMinerLoader
from langchain.document_loaders import PyPDFLoader

## 1.2. Initial variable setup

In [3]:
## Initial variable setup
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-large")
db_directory = "./data/chroma_semantic"
USE_Anthropic = True

if USE_Anthropic:
    llm = ChatAnthropic(model_name="claude-3-sonnet-20240229", temperature=0)
else:
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0) # context window size 16k for GPT 3.5 Turbo

collection_list=[
    "ASOP_life",
    "Bermuda",
    "CFT",
    "PBR",
    "VM21",
    "VM22",
    "Asset",
    "IFRS17"
]
#collection_list = ["PBR"] # for testing

# 2. Load PDF files and convert to a vector DB
## 2.1. Define functions

In [11]:
# Define a function to load and extract text from PDFs in a folder
def get_file_name(source_path):
    return source_path.split('/')[-1]

def load_pdfs_from_folder(folder_path, loader_option):
    # Get a list of PDF files in the specified folder
    pdf_files = glob.glob(f"{folder_path}/*.pdf")
    docs = []
    for pdf_file in pdf_files:
        file_name = get_file_name(pdf_file)
        
        if loader_option == 1:
            # Load the PDF file using the PyPDFLoader
            loader = PyPDFLoader(pdf_file)
        elif loader_option == 2:
            # PyPDFium2Loader is known to be faster than PyPDFLoader
            loader = PyPDFium2Loader(pdf_file)
        elif loader_option == 3:
            # PyMuPDFLoader is known to be general purpose, rich metadata
            loader = PyMuPDFLoader(pdf_file)
        elif loader_option == 4:
            # Allows automated concatenate pages
            loader = PDFMinerLoader(pdf_file, concatenate_pages=True)
        
        loaded_docs = loader.load()
        
        for doc in loaded_docs:
            doc.metadata['source'] = file_name
        
        docs.extend(loaded_docs)
    return docs

def pdf_to_md(folder_path, download_path, loader_option):
    # Get a list of PDF files in the specified folder
    pdf_files = glob.glob(f"{folder_path}/*.pdf")
    for pdf_file in pdf_files:
        file_name = get_file_name(pdf_file)
        base_name = file_name.replace('.pdf', '')
        
        if loader_option == 1:
            # Load the PDF file using the PyPDFLoader
            loader = PyPDFLoader(pdf_file)
        elif loader_option == 2:
            # PyPDFium2Loader is known to be faster than PyPDFLoader
            loader = PyPDFium2Loader(pdf_file)
        elif loader_option == 3:
            # PyMuPDFLoader is known to be general purpose, rich metadata
            loader = PyMuPDFLoader(pdf_file)
        elif loader_option == 4:
            # Allows automated concatenate pages
            loader = PDFMinerLoader(pdf_file, concatenate_pages=True)
        elif loader_option == 5:
            # Use Mathpix OCR to load formula, tables
            # may be slower, but higher quality than all above
            # Require Mathpix API ID - 3 cents per pdf page
            loader = MathpixPDFLoader(pdf_file)
        
        loaded_docs = loader.load()
        
        for i, doc in enumerate(loaded_docs):
            doc.metadata['source'] = file_name
            if loader_option > 3:
                md_file_name = f"{download_path}/{base_name}.md"
            else:
                md_file_name = f"{download_path}/{base_name}{i+1:03d}.md"
            with open(md_file_name, 'w', encoding='utf-8') as md_file:
                md_file.write(doc.page_content)

def load_mds_from_folder(folder_path):
    # Get a list of md files in the specified folder
    md_files = glob.glob(f"{folder_path}/*.md")
    docs = []
    for md_file in md_files:
        file_name = get_file_name(md_file)
        base_name = file_name.replace('.md', '')
        pdf_file_name = f"{base_name}.pdf"
        
        loader = UnstructuredMarkdownLoader(md_file)
        loaded_docs = loader.load()
        
        for doc in loaded_docs:
            doc.metadata['source'] = pdf_file_name
        
        docs.extend(loaded_docs)
    return docs


## 2.2. Convert PDFs to markdown files and then convert to vector database

In [16]:
############################################################################
# Run only to convert pdf to markdown files
############################################################################

collection_list = ["GAAP"]
for collection_name in collection_list:
    # Put new files in the upload subfolder
    folder_path = './data/upload/pdf/'+collection_name
    download_path = './data/upload/md/'+collection_name
    os.makedirs(download_path, exist_ok=True)

    # Use loader option 5 to use Mathpix OCR to load formula, tables
    pdf_to_md(folder_path, download_path, loader_option = 5)

Status: loaded, waiting for processing to complete
Status: split, waiting for processing to complete
Status: loaded, waiting for processing to complete
Status: loaded, waiting for processing to complete
Status: split, waiting for processing to complete
Status: split, waiting for processing to complete
Status: loaded, waiting for processing to complete
Status: loaded, waiting for processing to complete
Status: split, waiting for processing to complete
Status: split, waiting for processing to complete
Status: split, waiting for processing to complete
Status: loaded, waiting for processing to complete
Status: split, waiting for processing to complete
Status: split, waiting for processing to complete


In [17]:
############################################################################
# Run to load markdown files to vector database
############################################################################
#collection_list=[
    # "ASOP_life",
    # "Bermuda",
    # "CFT",
    # "PBR",
    # "VM21",
    # "VM22",
    # "Asset",
    # "IFRS17",
#]

for collection_name in collection_list:
    # Put new files in the upload subfolder
    folder_path = './data/upload/md/'+collection_name

    # Call the function to load and extract text from PDFs in the specified folder
    docs = load_mds_from_folder(folder_path)
    
    # Create a text splitter object with specified parameters
    # text_splitter = RecursiveCharacterTextSplitter(
    #     chunk_size=1000, # 1000 splits a page into roughly 3 chunks
    #     chunk_overlap=200,
    #     length_function=len,)

    # Use semantic chunker to increase meaningfulness of the chunks
    text_splitter = SemanticChunker(embeddings_model)

    # Split the documents into chunks using the text splitter
    splits = text_splitter.split_documents(docs)

    # Create a Chroma vector database from the document splits
    Chroma.from_documents(
        documents=splits, 
        embedding=embeddings_model, 
        persist_directory=db_directory,
        collection_name=collection_name,
    )

## 2.3. (NOT USED) Convert PDFs to vector database directly

In [ ]:
############################################################################
# Run to load pdf files to vector database
############################################################################

for collection_name in collection_list:
    # Put new files in the upload subfolder
    folder_path = './data/upload/pdf/'+collection_name

    # Call the function to load and extract text from PDFs in the specified folder
    docs = load_pdfs_from_folder(folder_path, loader_option = 1)
    
    # Create a text splitter object with specified parameters
    # text_splitter = RecursiveCharacterTextSplitter(
    #     chunk_size=1000, # 1000 splits a page into roughly 3 chunks
    #     chunk_overlap=200,
    #     length_function=len,)

    # Use semantic chunker to increase meaningfulness of the chunks
    text_splitter = SemanticChunker(embeddings_model)

    # Split the documents into chunks using the text splitter
    splits = text_splitter.split_documents(docs)

    # Create a Chroma vector database from the document splits
    Chroma.from_documents(
        documents=splits, 
        embedding=embeddings_model, 
        persist_directory=db_directory,
        collection_name=collection_name,
    )

# 3. For test purposes
## 3.1. Define vector store from vector database 

In [78]:
## a user may choose different collection name from the list
# ["ASOP_life", "Bermuda", "CFT", "VM21", "VM22", "Asset", "IFRS17"]
collection_name = collection_list[0] 

# Get a Chroma vector database with specified parameters
vectorstore = Chroma(embedding_function=embeddings_model, 
                     persist_directory=db_directory,
                     collection_name=collection_name)

## 3.2. Retrieve from the vector store

In [74]:
## Retrieve and RAG chain
# Create a retriever using the vector database as the search source
# You may choose a specific document to filter the search
retriever = vectorstore.as_retriever(search_type="mmr", 
                                     search_kwargs={
                                        'k': 6, 
                                        'lambda_mult': 0.5,
                                        # 'filter': {'source': '201611-Guidance-Notes-for-Commercial-Insurers-and-Groups-Statutory-Reporting-Regime-30-Nov-2016.pdf'}
                                        }
                                    ) 
# Use MMR (Maximum Marginal Relevance) to find a set of documents that are both similar to the input query and diverse among themselves
# Increase the number of documents to get, and increase diversity (lambda mult 0.5 being default, 0 being the most diverse, 1 being the least)

# Load the RAG (Retrieval-Augmented Generation) prompt
qa_system_prompt = """You are a helpful assistant to help actuaries with question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
ASOP or asop means Actuarial Standards of Practice. \
CFT means Cash Flow Testing. AAT means Asset Adequacy Testing. \
BMA means Bermuda Monetary Authority. \
SBA means scenario-based approach. BEL means best estimate liabilities.\
After you answer, provide the sources you used to answer the question. \
If you don't know the answer, just say that you don't know. \

{context}"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        ("human", "{question}"),
    ]
)

# Define a function to format the documents with their sources and pages
def format_docs_with_sources(docs):
    formatted_docs = "\n\n".join(doc.page_content for doc in docs)
    #sources_pages = "\n".join(f"{doc.metadata['source']} (Page {doc.metadata['page'] + 1})" for doc in docs)
    sources_pages = "\n".join(f"{doc.metadata['source']})" for doc in docs)
    # Added 1 to the page number assuming 'page' starts at 0 and we want to present it in a user-friendly way

    return f"Documents:\n{formatted_docs}\n\nSources and Pages:\n{sources_pages}"

# Create a RAG chain using the formatted documents as the context
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs_with_sources(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

# Create a parallel chain for retrieving and generating answers
rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

## 3.3. Generate Q&A Function

In [ ]:
def generate_output():
    # Prompt the user for a question on ASOP
    usr_input = input("What is your question on ASOP?: ")

    # Invoke the RAG chain with the user input as the question
    output = rag_chain_with_source.invoke(usr_input)

    # Generate the Markdown output with the question, answer, and context
    markdown_output = "### Question\n{}\n\n### Answer\n{}\n\n### Context\n".format(output['question'], output['answer'])

    last_page_content = None  # Variable to store the last page content
    i = 1 # Source indicator

    # Iterate over the context documents to format and include them in the output
    for doc in output['context']:
        current_page_content = doc.page_content.replace('\n', '  \n')  # Get the current page content
        
        # Check if the current content is different from the last one
        if current_page_content != last_page_content:
            #markdown_output += "- **Source {}**: {}, page {}:\n\n{}\n".format(i, doc.metadata['source'], doc.metadata['page'], current_page_content)
            markdown_output += "- **Source {}**: {}:\n\n{}\n".format(i, doc.metadata['source'], current_page_content)
            i = i + 1
        last_page_content = current_page_content  # Update the last page content
    
    # Display the Markdown output
    display(Markdown(markdown_output))

### Example questions related to ASOPs
- explain ASOP No. 14
- How are expenses relfected in cash flow testing based on ASOP No. 22?
- What is catastrophe risk?
- When do I update assumptions?
- What should I do when I do not have credible data to develop non-economic assumptions?

In [ ]:
generate_output()

# 4. References
- https://www.actuarialstandardsboard.org/standards-of-practice/
- https://python.langchain.com/docs/use_cases/question_answering/quickstart
- https://python.langchain.com/docs/use_cases/question_answering/sources
- https://python.langchain.com/docs/integrations/text_embedding/
- https://python.langchain.com/docs/integrations/vectorstores/chroma
- https://docs.gpt4all.io/gpt4all_python_embedding.html#gpt4all.gpt4all.Embed4All
- https://chat.langchain.com/
- https://api.python.langchain.com/en/latest/vectorstores/langchain_community.vectorstores.chroma.Chroma.html

# 5. Management of the vector database

In [4]:
client = chromadb.PersistentClient(path=db_directory)

In [80]:
#client.delete_collection(name="VM20") # Delete a collection and all associated files

In [5]:
collection = client.get_collection(name="PBR") 
collection.count()


463

In [6]:
collection.peek(1)

{'ids': ['000bfa27-4837-4201-a0c8-d191a82eb44d'],
 'embeddings': [[0.012384675443172455,
   -0.014510467648506165,
   -0.023512810468673706,
   0.035458557307720184,
   -0.01611987315118313,
   0.03015698492527008,
   0.008365465328097343,
   0.020810386165976524,
   0.014768660999834538,
   0.05346323922276497,
   -0.021120218560099602,
   0.031688932329416275,
   0.011816649697721004,
   -0.02143004909157753,
   -0.024046409875154495,
   -0.004122487269341946,
   -0.02196364849805832,
   0.012840816751122475,
   -0.007289659697562456,
   -0.02833241969347,
   0.013727281242609024,
   0.0132195008918643,
   -0.010026508942246437,
   0.032394662499427795,
   0.0043290420435369015,
   0.026077531278133392,
   0.01187689509242773,
   0.017066581174731255,
   0.0338921844959259,
   0.0015147344674915075,
   0.004509777296334505,
   0.018503857776522636,
   -0.0005680253962054849,
   -0.046819064766168594,
   -0.016860026866197586,
   0.008133091032505035,
   0.02191201038658619,
   0.0123

In [88]:
#collection.modify(name="PBR") rename

In [10]:
collection.get(
    where={"source": "VA_PN_Supplement_Exposure_Draft.pdf"}
)

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None}

In [9]:
collection.delete(
    #ids=["id1", "id2", "id3",...],
    where={"source": "VA_PN_Supplement_Exposure_Draft.pdf"}
)